In [1]:
from shared_lib.spark import get_spark_session

spark = get_spark_session(app_name="dataops-crypto-cloud", master=True, local=True, minio=True, hive=True, jars=True)

26/02/26 16:29:07 WARN Utils: Your hostname, Nguyens-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.23 instead (on interface en0)
26/02/26 16:29:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/anhtu/.ivy2/cache
The jars for the packages stored in: /Users/anhtu/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.iceberg#iceberg-aws-bundle added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-19766615-e76a-4841-b609-be8da58c42fa;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/anhtu/.pyenv/versions/3.11.11/envs/spark/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.7.1 in central
	found org.apache.iceberg#iceberg-aws-bundle;1.7.1 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 108ms :: artifacts dl 4ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.apache.iceberg#iceberg-aws-bundle;1.7.1 from central in [default]
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.7.1 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	-------------------------------

In [2]:
import os
from shared_lib.spark import create_database

test_db = f"hive_catalog.{os.getenv('TRANSFORM_DB')}"
test_db_location = f"s3a://{os.getenv('DATA_LAKE_BUCKET')}/transform_zone/"


In [3]:
from shared_lib.spark import database_exists


if database_exists(spark, test_db):
    print("Database exists")
else:    
    create_database(spark, test_db, test_db_location)
    print("Database created")


Database exists


In [4]:
spark.sql("SHOW DATABASES in hive_catalog").show()

+------------+
|   namespace|
+------------+
|     default|
|    test_db2|
|    test_db3|
|transform_db|
+------------+



In [6]:
spark.sql(f"select * from {test_db}.pattern_two where pattern is not null").show()

+--------+-------------------+----------------+----------+----------+---------+-----------+--------+----------------+------------+-------+------+------+---------+-----------------+
|group_id|         group_date|       open_time|open_price|high_price|low_price|close_price|  volume|      close_time|landing_date| symbol|  ema7| ema20|    trend|          pattern|
+--------+-------------------+----------------+----------+----------+---------+-----------+--------+----------------+------------+-------+------+------+---------+-----------------+
| 1954413|2025-09-27 11:15:00|1758971700405060|    0.7837|    0.7853|   0.7837|      0.785|121594.4|1758972591323187|  2025-09-27|ADAUSDT|0.7835| 0.784|downtrend|bullish engulfing|
| 1954445|2025-09-27 19:15:00|1759000502967967|    0.7803|    0.7823|     0.78|     0.7821| 95233.0|1759001384883232|  2025-09-27|ADAUSDT|0.7808|0.7817|downtrend|bullish engulfing|
+--------+-------------------+----------------+----------+----------+---------+-----------+----